In [60]:
# import Pkg;
# Pkg.add("PyPlot")
# Pkg.add("CSV"); 
# Pkg.add("DataFrames");

In [61]:
using CSV
using DataFrames
using LinearAlgebra
using Plots

In [62]:
function get_actual_values_from_cumulatives(cumulative_vector)
    shifted_values = zeros(Float64, size(cumulative_vector)[1]+1)
    for (index, value) in enumerate(cumulative_vector)
       shifted_values[index+1] = value  
    end

    final_values = zeros(Float64, size(cumulative_vector)[1])
    for (index, value) in enumerate(cumulative_vector)
       final_values[index] = -shifted_values[index]+cumulative_vector[index]  
    end
    return final_values
end

get_actual_values_from_cumulatives (generic function with 1 method)

Read data. Data downloaded from https://github.com/pcm-dpc/COVID-19/blob/master/dati-andamento-covid19-italia.md

In [63]:
# infected_cases_data_path = "data/countries/time_series_covid19_confirmed_global.csv"
# recovered_cases_data_path = "data/countries/time_series_covid19_recovered_global.csv"
# deceased_cases_data_path = "data/countries/time_series_covid19_deaths_global.csv"

# infected_total = CSV.read(infected_cases_data_path, DataFrame)
# infected_US = infected_total[infected_total[!, :"Country/Region"] .== "US", :]
# infected = select(infected_US, Not([:"Province/State", :"Lat", :"Long", :"Country/Region"])) #remove columns that are not required

# recovered_total = CSV.read(recovered_cases_data_path, DataFrame)

# recovered_US = recovered_total[recovered_total[!, :"Country/Region"] .== "US", :]
# recovered = select(recovered_US, Not([:"Province/State", :"Lat", :"Long", :"Country/Region"])) #remove columns that are not required


# deceased_total = CSV.read(deceased_cases_data_path, DataFrame)
# deceased_US = deceased_total[deceased_total[!, :"Country/Region"] .== "US", :]
# deceased = select(deceased_US, Not([:"Province/State", :"Lat", :"Long", :"Country/Region"]))#remove columns that are not required


# # #convert to matrices

# infected = Matrix(infected)'[1:327]
# recovered = Matrix(recovered)'[1:327] #considering only until Dec 13 2020.
# deceased = Matrix(deceased)'[1:327]
# P = 332902734

# ω_file="data/usa_ω.csv"
# α_file="data/usa_α.csv"
# error_file="data/usa_error.csv"

In [64]:
data_path = "data/countries/italy.csv"

data = CSV.read(data_path, DataFrame)
# totale_positivi,dimessi_guariti,deceduti 
# (Total amount of current positive cases (Hospitalised patients + Home confinement))
# recovered
# deceased

infected = data[!, :"totale_positivi"][1:36]
recovered = data[!, :"dimessi_guariti"][1:36]
deceased = data[!, :"deceduti"][1:36]
P = 59583924

ω_file="data/italy_ω.csv"
α_file="data/italy_α.csv"
error_file="data/italy_error.csv"
print("Done")

Done

Algorithm:

Equations:
    I(t) = αĨ(t)

    R(t) = αR̃(t)

    S(t) = αS̃(t)

where I(t), R(t) and S(t) are the actual number of infected, recovered and susceptible population at time t and α≥1;
   
    S(t₀) = ωP

where ωϵ[0,1] and P is the total population.

S̃(t + 1) = S̃(t) - β * (S̃(t) * Ĩ(t)) ÷ (S̃(t) + Ĩ(t))
Ĩ(t + 1) = Ĩ(t) + β * (S̃(t) * Ĩ(t)) ÷ (S̃(t) + Ĩ(t)) - γ * Ĩ(t) - ν * Ĩ(t)
R̃(t + 1) = R̃(t) + γ * Ĩ(t)
D(t + 1) = D(t) + α * ν * Ĩ(t)

where
    I(t) = αĨ(t)

    R(t) = αR̃(t)

    S(t) = αS̃(t)

where I(t), R(t) and S(t) are the actual number of infected, recovered and susceptible population at time t and α≥1;
   
    S(t₀) = ωP

where ωϵ[0,1] and P is the total population.

    S̃(t) = (ω ÷ α) * P - Ĩ(t) - R̃(t) - D̃(t)

    Δ(t) = [Ĩ(t + 1)-Ĩ(t); R̃(t + 1)-R̃(t); D̃(t + 1)-D̃(t)]
    Δ(t) = Φ_ωα(t) = [S̃(t)-β*(S̃(t)*Ĩ(t))÷(S̃(t)+Ĩ(t)) -Ĩ(t) -Ĩ(t)÷α; 0 Ĩ(t) 0; 0 0 Ĩ(t)]
    
    Δ̄ = [ρ^(Θ -t₀)Δ(t₀); ρ^(Θ -t₀-1)Δ(t₀+1); ρ^(Θ -t₀-2)Δ(t₀+2)...Δ(Θ)]
    Φ̄_ωα = [ρ^(Θ -t₀)Φ_ωα(t₀); ρ^(Θ -t₀-1)Φ_ωα(t₀+1); ρ^(Θ -t₀-2)Φ_ωα(t₀+2)...Φ_ωα(Θ)]

where ρϵ(0,1)


Steps:
1. Input: data Ĩ(t), R̃(t), and D(t), maximum value ᾱ of α, weighting parameter ρ, and total population P

outputs -> ω⋆, α⋆, β⋆, γ⋆, ν̄⋆


In [65]:
#we have Ĩ(t) and R̃(t) and D(t) in infected, recovered and deceased dataframes.
	 #https://www.census.gov/popclock/
ᾱ = 101       # ᾱ*Infected + ᾱ*Recovered + ᾱ*Deceased < ω*Population
ρ = 0.9
ω = 0.005 
α_0 = 1
α_increment = 0.5
ω_increment = 0.005

num_α = Int64((ᾱ-α_0)/α_increment)+1
num_ω = Int64((1/ω_increment))

200

In [66]:
function get_susceptible(ω, α, P, Ĩ, R̃, D)
    population = zeros(Float64, size(Ĩ)[1])
    fill!(population, ω*P/α)
    return population-infected-recovered-deceased
end

get_susceptible (generic function with 1 method)

In [67]:
function invalid(vector_values)
    for (index, value) in enumerate(vector_values)
        if value < 0
            return true
        end
    end
    return false
end

invalid (generic function with 1 method)

In [68]:
function computeΔ̄(Ĩ, R̃, D, ρ)
    Θ = size(Ĩ)[1]
    
    ρmatrix = zeros(Float64, Θ)
    for i in 1:Θ
        power = Θ-i
        ρmatrix[i] = ρ^power
    end
    
    Δ̄ = zeros(Float64, 3Θ)
    Δ̄[1] = ρmatrix[1]*(Ĩ[1])
    Δ̄[2] = ρmatrix[1]*(R̃[1])
    Δ̄[3] = ρmatrix[1]*(D[1])
    for i in 1:Θ-1
        Δ̄[3*i+1] = ρmatrix[i]*(Ĩ[i+1]-Ĩ[i])
        Δ̄[3*i+2] = ρmatrix[i]*(R̃[i+1]-R̃[i])
        Δ̄[3*i+3] = ρmatrix[i]*(D[i+1]-D[i])
    end
    

    return Δ̄
end

computeΔ̄ (generic function with 1 method)

In [69]:
function computeΦ(ω, α, S̃, Ĩ)
    Θ = size(Ĩ)[1]
    
    ρmatrix = zeros(Float64, Θ)
    for i in 1:Θ
        power = Θ-i
        ρmatrix[i] = ρ^power
    end
    
    Φ = zeros(Float64, 3*Θ, 3)
    i=1
    j = 1
    while i<=Θ
        Φ[j] = ρmatrix[i]*(S̃[i]*Ĩ[i])/ (S̃[i]+Ĩ[i])
        Φ[3*Θ+j] = -1*ρmatrix[i]*Ĩ[i]
        Φ[6*Θ+j] = -1*ρmatrix[i]*Ĩ[i]/α
        Φ[j+1] = 0
        Φ[3*Θ+j+1] = ρmatrix[i]*Ĩ[i]
        Φ[6*Θ+j+1] = 0
        Φ[j+2] = 0
        Φ[3*Θ+j+2] = 0
        Φ[6*Θ+j+2] = ρmatrix[i]*Ĩ[i]
        i+=1
        j+=3
    end
    return Φ
    
end

# S̃ = get_susceptible(0.5, 2, P, infected, recovered, deceased)
# computeΦ(0.5, 2, S̃, infected)


computeΦ (generic function with 1 method)

In [70]:
function computeParameters(Φ̄, Δ̄)
    # @time pseudo_inv = pinv(Φ̄)    # Uncomment to check time taken.
    pseudo_inv = pinv(Φ̄)
    params = pseudo_inv * Δ̄
    return params
end

computeParameters (generic function with 1 method)

In [71]:
function error_(Δ̄, Φ̄, params)
    param_matrix = zeros(Float64, 3, 1)
    param_matrix[1] = params[1]
    param_matrix[2] = params[2]
    param_matrix[3] = params[3]
    
    diff = Δ̄-(Φ̄*param_matrix)
    return norm(diff)^2
end

# M = [1.5 1.3; 1.2 1.9]
# N = [2,4]
# params = computeParameters(M, N)
# error_(N, M, params)
# # print(params)


error_ (generic function with 1 method)

In [72]:
function sir_model(Ĩ, R̃, D, P, ᾱ, ρ,ω,α_0,α_increment,ω_increment)
    e = Inf
    Δ̄ = computeΔ̄(Ĩ, R̃, D, ρ)
    ω_best = Inf
    α_best = Inf
    β_best = Inf
    γ_best = Inf
    ν̃_best = Inf
    
    num_α = Int64((ᾱ-α_0)/α_increment)+1
    num_ω = Int64((1/ω_increment))
    num_solve_points = (num_α*num_ω)
    loss_vals = zeros(Float64, num_ω, num_α)
    
    ω_store = zeros(Float64, num_ω, num_α)
    α_store = zeros(Float64, num_ω, num_α)
    β_store = zeros(Float64, num_ω, num_α)
    γ_store = zeros(Float64, num_ω, num_α)
    ν̃_store = zeros(Float64, num_ω, num_α)
    
    i = j = 1
    while ω ≤ 1
        α = α_0
        j=1
        while α ≤ ᾱ
            S̃ = get_susceptible(ω, α, P, Ĩ, R̃, D)
            if invalid(S̃)
                ω_store[i,j] = ω
                α_store[i,j] = α
                β_store[i,j] = 0
                γ_store[i,j] = 0
                ν̃_store[i,j] = 0
                loss_vals[i,j] = 0
                α+=α_increment
                j+=1
                continue
            end
            
            Φ̄ = computeΦ(ω, α, S̃, Ĩ)
            params = computeParameters(Φ̄, Δ̄)
            β = params[1]
            γ = params[2]
            ν̃ = params[3]
            if(β<0 || γ<0 || ν̃<0)
                ω_store[i,j] = ω
                α_store[i,j] = α
                β_store[i,j] = 0
                γ_store[i,j] = 0
                ν̃_store[i,j] = 0
                loss_vals[i,j] = 0
                α+=α_increment
                j+=1
                continue
            end

            error_value = error_(Δ̄, Φ̄, params)
            if error_value < e
                e = error_value
                ω_best = ω
                α_best = α
                β_best = β
                γ_best = γ
                ν̃_best = ν̃
            end

            ω_store[i,j] = ω
            α_store[i,j] = α
            β_store[i,j] = β
            γ_store[i,j] = γ
            ν̃_store[i,j] = ν̃
            loss_vals[i,j] = error_value
            
            α+=α_increment
            j+=1
        end
        ω+=ω_increment
        i+=1
    end
    return ω_best,α_best,β_best,γ_best,ν̃_best, e, ω_store, α_store, β_store, γ_store, ν̃_store, loss_vals
end

sir_model (generic function with 1 method)

In [73]:
ω_, α_, β_, γ_, ν̃_, least_loss, ω_store, α_store, β_store, γ_store, ν̃_store, loss_vals = sir_model(infected, recovered, deceased, P, ᾱ, ρ,ω,α_0,α_increment,ω_increment)
println("Done.")

Done.


In [74]:
function meshgrid(xin,yin)
    nx=length(xin)
    ny=length(yin)
    xout=zeros(ny,nx)
    yout=zeros(ny,nx)
    for jx=1:nx
        for ix=1:ny
            xout[ix,jx]=xin[jx]
            yout[ix,jx]=yin[ix]
        end
    end
    return (x=xout, y=yout)
end

meshgrid (generic function with 1 method)

In [75]:
function plot_loss_curve(ω_, α_, β_, γ_, ν̃_, least_loss, ω_store, α_store, β_store, γ_store, ν̃_store, loss_vals)
    # ω_s,α_s = meshgrid(vec(ω_store), vec(α_store))
    surface(reshape(vec(ω_store),(num_ω,num_α)), reshape(vec(α_store),(num_ω,num_α)), reshape(vec(loss_vals),(num_ω,num_α)))
    # plot(x, y, z, zcolor = reverse(z), m = (10, 0.8, :blues, Plots.stroke(0)), leg = false, cbar = true, w = 5)
    # scatter!(ω_, α_, least_loss, w = 10)
end

plot_loss_curve (generic function with 1 method)

In [76]:
function plot_loss_tri(ω_, α_, β_, γ_, ν̃_, least_loss, ω_store, α_store, β_store, γ_store, ν̃_store, loss_vals)
    # ω_s,α_s = meshgrid(vec(ω_store), vec(α_store))
    PyPlot.plot_trisurf(vec(ω_store), vec(α_store), vec(loss_vals))
    # plot(x, y, z, zcolor = reverse(z), m = (10, 0.8, :blues, Plots.stroke(0)), leg = false, cbar = true, w = 5)
    # scatter!(ω_, α_, least_loss, w = 10)
end

plot_loss_tri (generic function with 1 method)

In [77]:
println(typeof(vec(ω_store)),size(vec(α_store)),size(vec(loss_vals)))

Vector{Float64}(40200,)(40200,)


In [78]:
# pyplot()
# plot_loss_curve(ω_, α_, β_, γ_, ν̃_, least_loss, ω_store, α_store, β_store, γ_store, ν̃_store, loss_vals)

In [79]:
println(ω_," ", α_, β_, γ_, ν̃_,least_loss)

0.19000000000000009 87.00.164091752893794130.0142470169646651060.0106486610131756222.6494983204609775e6


In [80]:
println(ω_," ", α_, β_, γ_, ν̃_," ",least_loss)

0.19000000000000009 87.00.164091752893794130.0142470169646651060.010648661013175622 2.6494983204609775e6


In [81]:
# plot_loss_tri(ω_, α_, β_, γ_, ν̃_, least_loss, ω_store, α_store, β_store, γ_store, ν̃_store, loss_vals)

In [82]:
pyplot()
Plots.PyPlotBackend()

Plots.PyPlotBackend()

In [83]:
function store_data(infected_df,recovered_df,deceased_df)
    infected_cases_data_path = "data/preprocessed/countries/time_series_covid19_confirmed_global.csv"
    recovered_cases_data_path = "data/preprocessed/countries/time_series_covid19_recovered_global.csv"
    deceased_cases_data_path = "data/preprocessed/countries/time_series_covid19_deaths_global.csv"

    CSV.write(infected_cases_data_path, infected_df)
#     infected_US = infected_total[infected_total[!, :"Country/Region"] .== "US", :]
#     infected = select(infected_US, Not([:"Province/State", :"Lat", :"Long", :"Country/Region"])) #remove columns that are not required

    CSV.write(recovered_cases_data_path, recovered_df)
#     recovered_US = recovered_total[recovered_total[!, :"Country/Region"] .== "US", :]
#     recovered = select(recovered_US, Not([:"Province/State", :"Lat", :"Long", :"Country/Region"])) #remove columns that are not required

    CSV.write(deceased_cases_data_path, deceased_df)
#     deceased_US = deceased_total[deceased_total[!, :"Country/Region"] .== "US", :]
#     deceased = select(deceased_US, Not([:"Province/State", :"Lat", :"Long", :"Country/Region"]))#remove columns that are not required


end

store_data (generic function with 1 method)

In [84]:
using DelimitedFiles
writedlm( ω_file,  ω_store, ',')
writedlm( α_file,  α_store, ',')
writedlm( error_file,  loss_vals, ',')

In [85]:
#compute S̃, Ĩ and R̃ of all time t.


# if !isfile("storage/preprocessed_data.csv")
#     for t = 1:length(infected_total)
#         S̃[t] = [ω / α] * P - Ĩ[t] - R̃[t] - D̃[t]
#         Ĩ[t + 1] = Ĩ[t] + β * [[S̃[t] * Ĩ[t]] / [S̃[t] + Ĩ[t]]] - γ * Ĩ[t] - ν * Ĩ[t]
#         R̃[t + 1] = R̃[t] + γ * Ĩ[t]
#         D[t + 1] = D[t] + α * ν * Ĩ[t]
#     store_data(infected_df,recovered_df,deceased_df)
# end

# Store to avoid repeated computing.